Here I try the code

In [117]:
import scrapy
import requests
from bs4 import BeautifulSoup
from scrapy import Selector
from scrapy.crawler import CrawlerProcess

In [ ]:
ico = ["27201121","27201121"]
start_urls = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"

html = requests.get(url).content

html

In [63]:
def get_html(ico):
    url = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"
    html = requests.get(url).content
    sel = Selector(text=html)
    vypis = sel.xpath('./a/@href')

    print(vypis)

get_html(27201121)




[]


In [ ]:
ico = "27082440"
url = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"
url2 = "https://or.justice.cz/ias/ui/rejstrik-firma.vysledky?subjektId=701502&typ=PLATNY"
html = requests.get(url2).content
soup = BeautifulSoup(html, 'lxml')

match = soup.find(text="Obchodní firma: ").findNext('span')
match2 = match.findNext('span')


match3 = soup.find(text="Identifikační číslo: ").findNext('span').text

match4 = soup.findAll(class_="nounderline")
match4[5].findNext('span')
print(match4)



In [ ]:
filter_list=[]

for article in soup.findAll(class_="nounderline"):
    article_text = article.text
    if not article_text:
        pass
    else: 
        filter_list.append(article)

filter_list


In [241]:
filter_list[3].findNext('span').text

'27082440'

In [ ]:
ico = "27201121"
url = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"
   
html = requests.get(url).content
sel = Selector(text=html)
vypis = sel.xpath("//li/a[contains(text(),'Výpis platných')]/@href").extract()

print(vypis)
